In [7]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [9]:
#initializing SQL engine
engine = create_engine('sqlite://///Users/allankapoor/Documents/Springboard/springboard/SQLFiles Tier 1/sqlite_db_pythonsqlite.db')
engine.table_names()

['Bookings', 'Facilities', 'Members']

In [10]:
# function for running sql query, outputs as pandas dataframe
def run_query(query):
    
    result = pd.read_sql_query(query, engine)
    return result

In [25]:
q = """
SELECT *
FROM Facilities
"""
facilities = run_query(q)
facilities

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [30]:
q = """
SELECT *
FROM Members
"""
members = run_query(q)
members.head()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [116]:
q = """
SELECT *
FROM Bookings
"""
bookings = run_query(q)
bookings

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1
...,...,...,...,...,...
4038,4038,8,29,2012-09-30 16:30:00,2
4039,4039,8,29,2012-09-30 18:00:00,1
4040,4040,8,21,2012-09-30 18:30:00,1
4041,4041,8,16,2012-09-30 19:00:00,1


<b>Q10:</b> Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [65]:
q = """
SELECT * FROM (
    WITH rev_table AS (
        SELECT
            b.facid,
            CASE WHEN memid = 0 THEN 'guest' ELSE 'member' END AS mem_type,
            SUM(slots) AS num_slots,
            f.membercost,
            f.guestcost
        FROM Bookings AS b

        INNER JOIN Facilities AS f
            ON b.facid = f.facid
        GROUP BY b.facid, mem_type
        )

    SELECT facid,
        SUM(CASE WHEN mem_type = 'guest' THEN num_slots * guestcost ELSE num_slots * membercost END) AS revenue

    FROM rev_table

    GROUP BY facid
    )
WHERE revenue < 1000
ORDER BY revenue DESC
"""

output = run_query(q)
output

,facid,revenue
0,8,270
1,7,240
2,3,180


<b>Q11:</b> Produce a report of members and who recommended them in alphabetic surname,firstname order

In [94]:
q = """
SELECT
    m1.surname AS mem_last,
    m1.firstname AS mem_first,
    m2.surname AS rec_first,
    m2.firstname AS rec_last
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid

ORDER BY mem_last, mem_first


"""
result = run_query(q)
result

,mem_last,mem_first,rec_first,rec_last
0,Bader,Florence,Stibbons,Ponder
1,Baker,Anne,Stibbons,Ponder
2,Baker,Timothy,Farrell,Jemima
3,Boothe,Tim,Rownam,Tim
4,Butters,Gerald,Smith,Darren
5,Coplin,Joan,Baker,Timothy
6,Crumpet,Erica,Smith,Tracy
7,Dare,Nancy,Joplette,Janice
8,Genting,Matthew,Butters,Gerald
9,Hunt,John,Purview,Millicent


<b>Q12:</b> Find the facilities with their usage by member, but not guests

Note - unclear from question whether the results are supposed to be split out by member (as I have done in the cell directly below) or if the question is really asking for the total usage of each facility by members (following cell).

In [104]:
q = """

WITH b AS(
    SELECT
        facid,
        memid,
        SUM(slots) AS num_slots
    FROM Bookings
    WHERE memid != 0
    GROUP BY facid, memid
    ORDER BY num_slots DESC
    )
SELECT
    f.name AS facility,
    m.surname AS last_n,
    m.firstname AS first_n,
    b.num_slots as bookings
FROM b
INNER JOIN Members AS m
    ON b.memid = m.memid
INNER JOIN Facilities AS f
    ON b.facid = f.facid
"""
result = run_query(q)
result

,facility,last_n,first_n,bookings
0,Badminton Court,Smith,Darren,432
1,Pool Table,Rownam,Tim,282
2,Massage Room 1,Rownam,Tim,176
3,Tennis Court 1,Butters,Gerald,171
4,Tennis Court 2,Boothe,Tim,168
...,...,...,...,...
197,Snooker Table,Farrell,David,2
198,Snooker Table,Purview,Millicent,2
199,Pool Table,Jones,Douglas,2
200,Pool Table,Owen,Charles,1


In [105]:
q = """

WITH b AS(
    SELECT
        facid,
        SUM(slots) AS num_slots
    FROM Bookings
    WHERE memid != 0
    GROUP BY facid
    ORDER BY num_slots DESC
    )
SELECT
    f.name AS facility,
    b.num_slots as bookings
FROM b
INNER JOIN Facilities AS f
    ON b.facid = f.facid
"""
result = run_query(q)
result

,facility,bookings
0,Badminton Court,1086
1,Tennis Court 1,957
2,Massage Room 1,884
3,Tennis Court 2,882
4,Snooker Table,860
5,Pool Table,856
6,Table Tennis,794
7,Squash Court,418
8,Massage Room 2,54


<b>Q13:</b> Find the facilities usage by month, but not guests

In [117]:
q = """

SELECT
    facid,
    strftime('%m', DATE(starttime)) AS month,
    SUM(slots) AS total_bookings

FROM Bookings

WHERE memid != 0

GROUP BY facid, month

ORDER BY facid, month

"""
result = run_query(q)
result

,facid,month,total_bookings
0,0,07,201
1,0,08,339
2,0,09,417
3,1,07,123
4,1,08,345
5,1,09,414
6,2,07,165
7,2,08,414
8,2,09,507
9,3,07,98
